In [6]:
import numpy as np 
import pandas as pd 
import seaborn as sb 
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier # default number of neighbors looked at is 5
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from umap import UMAP
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score 
import os 

#getting the data 
os.chdir('/Users/mishafaruki/Desktop/QBIO490/qbio_490_mishafaruki/analysis_data')

import cptac
#downloading 
cptac.download(dataset='Ccrcc')
ccrcc = cptac.Ccrcc()
ccrcc.list_data()

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.13. Please run 'pip install --upgrade cptac' to update it. (/Users/mishafaruki/miniconda3/envs/qbio_490_mishafaruki/lib/python3.9/threading.py, line 917)


Below are the dataframes contained in this dataset and their dimensions:

clinical
	194 rows
	171 columns
CNV
	110 rows
	19285 columns
followup
	352 rows
	27 columns
medical_history
	370 rows
	4 columns
methylation
	107 rows
	15885 columns
phosphoproteomics
	194 rows
	81550 columns
phosphoproteomics_gene
	194 rows
	6127 columns
proteomics
	194 rows
	11710 columns
somatic_mutation
	8350 rows
	3 columns
transcriptomics
	185 rows
	19275 columns


In [9]:
#Get the data you want 
clinical_data = ccrcc.get_clinical()
protein_data = ccrcc.get_proteomics()
rna_data = ccrcc.get_transcriptomics()

In [10]:
#extract data from stage i and stage iii 
stage_i_patients = clinical_data[clinical_data['tumor_stage_pathological'] == 'Stage I'].index
stage_iii_patients = clinical_data[clinical_data['tumor_stage_pathological'] == 'Stage III'].index

In [11]:
#protein
mean_stage_i_protein = protein_data.loc[stage_i_patients].mean()
mean_stage_iii_protein = protein_data.loc[stage_iii_patients].mean()
differential_expression_protein = mean_stage_iii_protein - mean_stage_i_protein
top_5_proteins = differential_expression_protein.abs().nlargest(5).index

In [12]:
#rna 
rna_data_log = np.log2(rna_data+1) 
mean_stage_i_rna = rna_data_log.loc[stage_i_patients].mean()
mean_stage_iii_rna = rna_data_log.loc[stage_iii_patients].mean()
differential_expression_rna = mean_stage_iii_rna - mean_stage_i_rna
top_5_rna = differential_expression_rna.abs().nlargest(5).index 

In [18]:
#selected features 
selected_features = pd.concat([protein_data[top_5_proteins], rna_data_log[top_5_rna]], axis=1)
selected_features.columns = selected_features.columns.astype(str)
cancer_stages = clinical_data['tumor_stage_pathological']

scaler = StandardScaler()
selected_features = selected_features.iloc[:, 5:]
patient_mask = selected_features.iloc[:, :].isna().sum(axis=1)==0
selected_features = selected_features.loc[patient_mask, :]
cancer_stages = cancer_stages.loc[patient_mask]

cancer_mask = cancer_stages.isna()
cancer_stages = cancer_stages.dropna()
selected_features = selected_features.loc[~cancer_mask, :]

selected_features_scaled = scaler.fit_transform(selected_features)
encoder = LabelEncoder()
cancer_stages_encoded = encoder.fit_transform(cancer_stages)

In [21]:
Xtrain, Xtest, ytrain, ytest = train_test_split(selected_features_scaled, cancer_stages_encoded, train_size=0.7)
Xtrain = pd.DataFrame(Xtrain, columns=selected_features.columns).dropna()
ytrain = ytrain[Xtrain.index] 

In [22]:
import warnings
warnings.filterwarnings('ignore')

models = {
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "MLPClassifier": MLPClassifier(),
    "GaussianNB": GaussianNB()
}

runs = 10
accuracies = {model_name: [] for model_name in models.keys()}

for _ in range(runs):
    for model_name, model in models.items():
        model.fit(Xtrain, ytrain)
        ypred = model.predict(Xtest)
        accuracy = accuracy_score(ytest, ypred)
        accuracies[model_name].append(accuracy)

mean_accuracies = {model_name: np.mean(accuracy_list) for model_name, accuracy_list in accuracies.items()}
best_model = max(mean_accuracies, key=mean_accuracies.get)

print("Mean accuracies:", mean_accuracies)
print("Best model:", best_model)

Mean accuracies: {'KNeighborsClassifier': 0.4242424242424242, 'DecisionTreeClassifier': 0.3484848484848485, 'MLPClassifier': 0.4, 'GaussianNB': 0.4545454545454545}
Best model: GaussianNB
